Imports

In [9]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

Setting up Directories

In [10]:
color_data_dir = "../data/colorData"
number_data_dir = "../data/numberData"
special_card_data_dir = "../data/specialCardData"
preprocessed_data_dir = "../data/preprocessedData"

Creating, Loading and Saving preprocessed Data

In [11]:
# Create directories for preprocessed data
os.makedirs(preprocessed_data_dir, exist_ok=True)


# Function to load, preprocess, and save images
def preprocess_and_save_images(directory, target_size=(300, 300)):
    images = []
    labels = []

    if os.path.isdir(directory):
        for label in os.listdir(directory):
            class_dir = os.path.join(directory, label)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    try:
                        img = Image.open(img_path).convert('RGB')
                        img = img.resize(target_size)
                        img_array = np.array(img, dtype=np.float32) / 255.0

                        images.append(img_array)
                        labels.append(label)
                    except Exception as e:
                        print(f"Error processing image {img_path}: {e}")
    
    # Encode labels as integers
    unique_labels = np.unique(labels)
    label_dict = {label: index for index, label in enumerate(unique_labels)}
    labels_encoded = np.array([label_dict[label] for label in labels], dtype=np.int64)

    return np.array(images, dtype=np.float32), labels_encoded, unique_labels

Preprocessing Images and Data Labels

In [12]:
color_images, color_labels, unique_color_labels = preprocess_and_save_images(color_data_dir)
number_images, number_labels, unique_number_labels = preprocess_and_save_images(number_data_dir)
special_card_images, special_card_labels, unique_special_labels = preprocess_and_save_images(special_card_data_dir)

Combining Images and Labels

In [13]:
all_images = np.concatenate([color_images, number_images, special_card_images], axis=0)
all_color_labels = np.concatenate([color_labels, np.full(len(number_labels), -1), np.full(len(special_card_labels), -1)])
all_number_labels = np.concatenate([np.full(len(color_labels), -1), number_labels, np.full(len(special_card_labels), -1)])
all_special_labels = np.concatenate([np.full(len(color_labels), -1), np.full(len(number_labels), -1), special_card_labels])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 2 has 1 dimension(s)

Ensure that all images and labels have the same length

In [ ]:
assert len(all_images) == len(all_color_labels) == len(all_number_labels) == len(all_special_labels), "Data lengths do not match!"

Split data into train and test sets

In [ ]:
X_train, X_test, y_train_color, y_test_color, y_train_number, y_test_number, y_train_special, y_test_special =  train_test_split(
                                                                                                                    all_images, 
                                                                                                                    all_color_labels,
                                                                                                                    all_number_labels,
                                                                                                                    all_special_labels,
                                                                                                                    test_size=0.2,
                                                                                                                    random_state=42
                                                                                                                )

Creating the Tenorflow Dataset

In [ ]:
# Create a TensorFlow dataset
def create_dataset(images, color_labels, number_labels, special_labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices(
        (images, {"color_output": color_labels, "number_output": number_labels, "special_output": special_labels})
    )
    
    dataset = dataset.shuffle(len(images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

Creating Training and Validation Datasets

In [ ]:
# Create training and validation datasets
train_dataset = create_dataset(X_train, y_train_color, y_train_number, y_train_special)
test_dataset = create_dataset(X_test, y_test_color, y_test_number, y_test_special)

Building the Model

In [ ]:
input_layer = Input(shape=(300, 300, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

Output Layers

In [ ]:
color_output = Dense(len(unique_color_labels), activation='softmax', name='color_output')(x)
number_output = Dense(len(unique_number_labels), activation='softmax', name='number_output')(x)
special_output = Dense(len(unique_special_labels), activation='softmax', name='special_output')(x)

Defining the Model

In [ ]:
model = Model(inputs=input_layer, outputs=[color_output, number_output, special_output])

Compiling the Model with Seperate Metrics for each Output

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'color_output': 'sparse_categorical_crossentropy',
                    'number_output': 'sparse_categorical_crossentropy',
                    'special_output': 'sparse_categorical_crossentropy'},
              metrics={'color_output': 'accuracy',
                       'number_output': 'accuracy',
                       'special_output': 'accuracy'})

Training the Model

In [ ]:
# Train the model
history = model.fit(train_dataset,
                    epochs=20, 
                    validation_data=test_dataset
                    )

Function to predict and display the result for a UNO card

In [ ]:
def predict_uno_card(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((300, 300))
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions
    color_prediction, number_prediction, special_prediction = model.predict(img_array)
    color_label = unique_color_labels[np.argmax(color_prediction)]
    number_label = unique_number_labels[np.argmax(number_prediction)]
    special_label = unique_special_labels[np.argmax(special_prediction)]

    print(f"Predicted color: {color_label}")
    print(f"Predicted number: {number_label}")
    print(f"Is special card: {special_label}")

Testing

In [ ]:
predict_uno_card("testing/red.jpg")